In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.decomposition import PCA
import xgboost as xgb

In [2]:
df = pd.read_csv('data_3k.csv')#, index_col='Unnamed: 0')
df.head()

,name,tempo,beats,chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc_1,...,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,label
0,Aaron Lewis - 'Country Boy' (Official Video).m...,125.000000,59,0.409949,0.044608,2684.678379,3327.070199,5616.759798,0.054827,150.374682,...,41.741827,-12.544442,28.144194,-13.241323,17.689333,-4.589490,3.226390,-4.043833,4.592962,Country
1,Aaron Lewis - 'Country Boy' (Official Video).m...,119.680851,58,0.431026,0.053247,2512.620073,3235.770491,4958.476826,0.053470,162.977125,...,29.105995,-7.695438,26.248572,-13.276758,15.348517,1.977714,0.815487,-3.722481,5.629056,Country
2,Aaron Lewis - 'Country Boy' (Official Video).m...,122.282609,59,0.412145,0.068782,2998.444102,3466.782972,6078.969294,0.068072,152.629035,...,36.643452,-11.062471,28.772055,-18.319343,11.178653,-6.798670,2.469059,-1.228891,10.052697,Country
3,Aaron Lewis - 'Country Boy' (Official Video).m...,122.282609,60,0.444166,0.066796,2655.326652,3391.723254,5624.541748,0.050296,158.986251,...,36.675408,-7.856390,27.634004,-10.692985,12.890847,-4.277223,6.024352,0.130230,1.666965,Country
4,Aaron Lewis - 'Country Boy' (Official Video).m...,122.282609,60,0.466898,0.076664,3007.087096,3441.825995,6096.982759,0.067290,153.416765,...,40.225236,-10.441213,34.054399,-17.073473,12.644855,-3.648381,3.667546,-2.283970,9.682853,Country


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3822 entries, 0 to 3821
Data columns (total 21 columns):
name           3822 non-null object
tempo          3822 non-null float64
beats          3822 non-null int64
chroma_stft    3822 non-null float64
rmse           3822 non-null float64
spec_cent      3822 non-null float64
spec_bw        3822 non-null float64
rolloff        3822 non-null float64
zcr            3822 non-null float64
mfcc_1         3822 non-null float64
mfcc_2         3822 non-null float64
mfcc_3         3822 non-null float64
mfcc_4         3822 non-null float64
mfcc_5         3822 non-null float64
mfcc_6         3822 non-null float64
mfcc_7         3822 non-null float64
mfcc_8         3822 non-null float64
mfcc_9         3822 non-null float64
mfcc_10        3822 non-null float64
mfcc_11        3822 non-null float64
label          3822 non-null object
dtypes: float64(18), int64(1), object(2)
memory usage: 627.1+ KB


In [3]:
#Assign int to genres
genre_dict = {
    'Country' : 0,
    'Classical' : 1,
    'Metal' : 2,
    'Jazz' : 3,
    'Hip_Hop' : 4,
    'Electronic' : 5
}

df['label'] = df['label'].map(genre_dict)
#df_test = df[(df['label'] != 0) & (df['label'] != 3) & (df['label'] != 5)]
X = df.drop(['name','label'],axis=1)
y = df['label']

In [34]:
#split data

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=10)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
scaler_train = StandardScaler()
X_train_scaled = scaler_train.fit_transform(X_train)
X_test_scaled = scaler_train.transform(X_test)
#pickle.dump(scaler_train, open('scaler.sav', 'wb'))

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all con

In [30]:
#SVC
from sklearn.svm import SVC
import pickle

clf = SVC(C=25,probability=True,decision_function_shape=('ovo'))
model = clf.fit(X_train_scaled,y_train)
print("Test fit: ",clf.predict_proba(X_test_scaled[2:3]))
filename = "model.sav"
pickle.dump(model, open(filename, 'wb'))

Test fit:  [[8.30856330e-04 9.97905798e-01 2.26884934e-04 9.28925413e-04
  3.14152856e-05 7.61203834e-05]]


In [29]:
loaded = pickle.load(open(filename, 'rb'))
print("Test: ", loaded.predict_proba(X_test_scaled[6:7]))

Test:  [[5.15041133e-02 9.26249295e-01 3.17114246e-03 1.11165239e-02
  7.53060004e-04 7.20586498e-03]]


In [ ]:
#Logisitic Regression
logreg = LogisticRegression(solver='lbfgs', multi_class='multinomial')
log_score = np.mean(cross_val_score(logreg,X_scaled,y,cv=3))
print("Mean Cross Validation Score: ", log_score)

In [ ]:
logreg.fit(X_train_scaled,y_train)
logreg_score = logreg.score(X_test_scaled,y_test)
print("Logistic Regression: ", logreg_score)

In [ ]:
#Decision Trees
dtc = DecisionTreeClassifier()
dtc_cv_score = np.mean(cross_val_score(dtc,X,y,cv=3))
dtc.fit(X_train,y_train)
dtc_acc_score = dtc.score(X_test,y_test)
print("Accuracy Score: ",dtc_acc_score)
print("Mean Cross Val Score: ",dtc_cv_score)

In [ ]:
param_grid = {
    'max_depth' : [],
    'min_samples_splt' : [],
    'min_samples_leaf' : [],
}

grid_search = GridSearchCV(dtc, param_grid, cv=3, return_train_score=True)
grid_search.fit(scaled_X_train,y_train)

In [ ]:
gs_train_score = np.mean(grid_search.cv_results_['mean_train_score'])
gs_test_score = grid_search.score(X_test,y_test)
print("Mean training score: ", gs_train_score)
print("Mean test score: ", gs_test_score)
print("Best params: ")
gs_grid_search.best_params_

In [ ]:
#Random Forests
#Bagged
bag = BaggingClassifier(n_estimators=100)
bag.fit(X_train, y_train)
print("Bagged score: ", bag.score(X_test,y_test))
#Random Forest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)
print("Random Forest Score: ", rf.score(X_test,y_test))

In [ ]:
#k-nearest neighbors
n = 1
p = 100
knn = KNeighborsClassifier(n_neighbors=n, p=p)
knn.fit(X_train_scaled,y_train)
print("K_Nearest Neightbors: ", knn.score(X_test_scaled,y_test))

In [ ]:
#XGBoost
#Scale?
# scaler = StandardScaler()
# scaler.fit(X)
# scaled_X = scaler.transform(X)

# pca = PCA(n_components=4)
# X_train_pca = pca.fit_transform(X_train_scaled)
# X_train,X_test,y_train,y_test = train_test_split(scaled_X,y,random_state=10)

#Classifier

xgb_clf = xgb.XGBClassifier(n_estimators=10)

xgb_clf.fit(X_train_scaled,y_train, verbose=True)
# train_preds = xgb_clf.predict(X_train_scaled)
# test_preds = xgb_clf.predict(X_test_scaled)
# train_acc = accuracy_score(y_train,train_preds)
# test_acc = accuracy_score(y_test,test_preds)
# print("XGBoost Scores")
# print("Train Score: ", train_acc * 100)
# print("Test Score: ", test_acc * 100)

In [3]:
from SongClassifier import SongClassifier
%load_ext autoreload
%autoreload 2

In [4]:
sc = SongClassifier()
#Lynard Skinard - Sweet Home Alabama
sc.predict('https://www.youtube.com/watch?v=ye5BuYf8q4o')

[youtube] ye5BuYf8q4o: Downloading webpage
[youtube] ye5BuYf8q4o: Downloading video info webpage
[download] Destination: mystery_song.webm
[download] 100% of 4.40MiB in 00:00
[ffmpeg] Destination: mystery_song.mp3
Deleting original file mystery_song.webm (pass -k to keep)


/anaconda3/lib/python3.7/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


test
test
-----------------------
Probability of Genre:
-----------------------
Country : 9.139999999999999%
Jazz : 31.619999999999997%
Metal : 42.86%
Hip_Hop : 9.93%
Electronic : 3.01%
Classical : 3.45%


In [47]:
#wiz khalifa - never been
sc.predict('https://www.youtube.com/watch?v=PKWFvbgkWls')

[youtube] PKWFvbgkWls: Downloading webpage
[youtube] PKWFvbgkWls: Downloading video info webpage
[download] Destination: mystery_song.webm
[download] 100% of 3.77MiB in 00:00
[ffmpeg] Destination: mystery_song.mp3
Deleting original file mystery_song.webm (pass -k to keep)
-----------------------
Probability of Genre:
-----------------------
Country : 2.69%
Jazz : 9.49%
Metal : 21.75%
Hip_Hop : 57.87%
Electronic : 7.779999999999999%
Classical : 0.42%


In [39]:
#Black Sabbath - Heaven and Hell
sc.predict('https://www.youtube.com/watch?v=idn50Xj_CiY')

[youtube] idn50Xj_CiY: Downloading webpage
[youtube] idn50Xj_CiY: Downloading video info webpage
[download] Destination: mystery_song.webm
[download] 100% of 6.38MiB in 00:00
[ffmpeg] Destination: mystery_song.mp3
Deleting original file mystery_song.webm (pass -k to keep)


/anaconda3/lib/python3.7/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


-----------------------
Probability of Genre:
-----------------------
Country : 53.37%
Jazz : 1.6099999999999999%
Metal : 17.59%
Hip_Hop : 21.349999999999998%
Electronic : 2.13%
Classical : 3.95%


In [36]:
#Rammstein - Feuer Frei
sc.predict('https://www.youtube.com/watch?v=ZkW-K5RQdzo')

[youtube] ZkW-K5RQdzo: Downloading webpage
[youtube] ZkW-K5RQdzo: Downloading video info webpage
[download] Destination: mystery_song.webm
[download] 100% of 2.79MiB in 00:00
[ffmpeg] Destination: mystery_song.mp3
Deleting original file mystery_song.webm (pass -k to keep)


/anaconda3/lib/python3.7/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


-----------------------
Probability of Genre:
-----------------------
Country : 3.91%
Jazz : 0.8999999999999999%
Metal : 60.42999999999999%
Hip_Hop : 0.5700000000000001%
Electronic : 27.61%
Classical : 6.59%


In [37]:
#Scorpions - Rock you like a hurricaine
sc.predict('https://www.youtube.com/watch?v=CGfKi6kpdTQ')

[youtube] CGfKi6kpdTQ: Downloading webpage
[youtube] CGfKi6kpdTQ: Downloading video info webpage
[youtube] CGfKi6kpdTQ: Downloading js player vflRqBT2o
[youtube] CGfKi6kpdTQ: Downloading js player vflRqBT2o
[download] Destination: mystery_song.webm
[download] 100% of 4.00MiB in 00:00
[ffmpeg] Destination: mystery_song.mp3
Deleting original file mystery_song.webm (pass -k to keep)


/anaconda3/lib/python3.7/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


-----------------------
Probability of Genre:
-----------------------
Country : 2.01%
Jazz : 11.5%
Metal : 50.849999999999994%
Hip_Hop : 16.900000000000002%
Electronic : 16.689999999999998%
Classical : 2.04%


In [51]:
#The Mars Volta - Eriatarka
sc.predict('https://www.youtube.com/watch?v=KQDcrFVN8mk')

[youtube] KQDcrFVN8mk: Downloading webpage
[youtube] KQDcrFVN8mk: Downloading video info webpage
[download] Destination: mystery_song.webm
[download] 100% of 5.90MiB in 00:00
[ffmpeg] Destination: mystery_song.mp3
Deleting original file mystery_song.webm (pass -k to keep)
-----------------------
Probability of Genre:
-----------------------
Country : 27.060000000000002%
Jazz : 0.38999999999999996%
Metal : 56.36%
Hip_Hop : 10.530000000000001%
Electronic : 3.36%
Classical : 2.3%


In [38]:
#Fare Thee Well - Marcus Mumford & Oscar Isaac
sc.predict('https://www.youtube.com/watch?v=SQyPVBtLXk0')

[youtube] SQyPVBtLXk0: Downloading webpage
[youtube] SQyPVBtLXk0: Downloading video info webpage
[download] Destination: mystery_song.webm
[download] 100% of 2.85MiB in 00:00
[ffmpeg] Destination: mystery_song.mp3
Deleting original file mystery_song.webm (pass -k to keep)


/anaconda3/lib/python3.7/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


-----------------------
Probability of Genre:
-----------------------
Country : 42.11%
Jazz : 21.32%
Metal : 1.68%
Hip_Hop : 30.740000000000002%
Electronic : 1.28%
Classical : 2.87%
